<a href="https://colab.research.google.com/github/hannahbhchou/song-recommendation/blob/main/Lyrics_Topic_Recommeder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preprocessing Lyrics Text

In [ ]:
import numpy as np
import pandas as pd
import time
import csv
import random
import re
import os

In [101]:
import warnings

In [102]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [91]:
lyrics_url = "https://gist.githubusercontent.com/hannahbhchou/6c1608f211a85d4d013d95a51c32a3f8/raw/833566eb910c086081b3b06951d2dc789b095c5d/song_lyrics_final.csv"

In [94]:
lyrics_df = pd.read_csv(url, names= ["URL", "Lyrics"])

In [95]:
lyrics_df.head()

,URL,Lyrics
0,https://genius.com/The-beatles-let-it-be-lyrics,"When I find myself in times of trouble, Mother..."
1,https://genius.com/The-beatles-come-together-l...,Shoot me\nShoot me\nShoot me\nShoot me\nHere c...
2,https://genius.com/The-beatles-yesterday-lyrics,Yesterday\nAll my troubles seemed so far away\...
3,https://genius.com/The-beatles-something-lyrics,Something in the way she moves\nAttracts me li...
4,https://genius.com/The-beatles-here-comes-the-...,"Here comes the sun, doo da doo doo\nHere comes..."


In [ ]:
#remove non-English text using unicode
lyrics_df['Lyrics'] = lyrics_df["Lyrics"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))

In [ ]:
#remove newline commands
lyrics_df["Lyrics"] = lyrics_df["Lyrics"].str.replace('\n',' ')

In [ ]:
lyrics_df['Lyrics'] = lyrics_df['Lyrics'].map(lambda x: x.lower())

In [ ]:
#remove the snippets such as [chorus]
lyrics_df['Lyrics'] = lyrics_df['Lyrics'].map(lambda x: re.sub(r'[\(\[].*?[\)\]]', '', x))

In [ ]:
#remove punctuations
lyrics_df['Lyrics'] = lyrics_df['Lyrics'].map(lambda x: re.sub('[,!+?:"()]', '', x))

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
np.random.seed(2018)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words = stopwords.words("english")

In [ ]:
added_words = ["oh","yeah","i'm","ooh","woo","ya","hoo", "hmm","ooooooohhh", "na", "lalala", "ah", "hey", "la", "uh", ]

In [ ]:
for x in added_words:
  stop_words.append(x)

In [ ]:
def remove_stopwords(raw_text):

    words = raw_text.split()
    not_stop_words = [w for w in words if not w in stop_words]
  
    return( " ".join(not_stop_words))

In [ ]:
lyrics_df['Lyrics'] = lyrics_df['Lyrics'].apply(lambda x: remove_stopwords(x))

### Lemmatizing & Stemming Text

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#testing lemmatizer
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [89]:
#testing stemmer
stemmer = SnowballStemmer('english')
original_words = ['candies', 'babies', 'dies', 'shoot' ,'died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating','sensational']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,candies,candi
1,babies,babi
2,dies,die
3,shoot,shoot
4,died,die
5,agreed,agre
6,owned,own
7,humbled,humbl
8,sized,size
9,meeting,meet


In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
#using the song https://genius.com/Lady-gaga-and-blackpink-sour-candy-lyrics to test
sample = lyrics_df.iloc[342,1]

In [ ]:
print('original document: ')
words = []
for word in sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(sample))

original document: 
['sou-sour', 'candy', 'sour', 'candy', 'sweet', 'get', 'little', 'angry', 'sour', 'candy', 'super', 'psycho', 'make', 'crazy', 'turn', 'lights', 'low', 'sour', 'candy', 'ask', 'nice', "i'll", 'extra', 'mean', 'wanna', 'fix', "let's", 'break', 'uh-huh', 'uh-huh', 'hard', 'outside', 'give', 'time', 'could', 'make', 'time', 'love', 'hard', 'outside', 'see', 'inside', 'inside', 'inside', 'might', 'messed', 'know', "what's", 'want', 'real', 'taste', 'least', 'fake', 'come', 'come', 'unwrap', 'come', 'come', 'unwrap', "i'll", 'show', "what's", 'close', 'eyes', 'peek', 'undressing', 'unwrap', 'sour', 'candy', 'come', 'come', 'unwrap', 'come', 'come', 'unwrap', 'come', 'sour', 'candy', 'hard', 'outside', 'give', 'time', 'could', 'make', 'time', 'love', 'hard', 'outside', 'see', 'inside', 'inside', 'inside', 'sour', 'candy', 'sweet', "'til", 'get', 'little', 'angry', 'sour', 'candy', 'super', 'psycho', 'make', 'crazy', 'turn', 'lights', 'low', 'sour', 'candy', 'take', 'bite'

In [ ]:
processed_lyrics = lyrics_df["Lyrics"].map(preprocess)

In [ ]:
processed_lyrics[:10]

0    [time, troubl, mother, mari, come, speak, word...
1    [shoot, shoot, shoot, shoot, come, flat, come,...
2    [yesterday, troubl, away, look, stay, believ, ...
3    [move, attract, like, lover, woo, want, leav, ...
4    [come, come, right, littl, darl, long, cold, l...
5    [jude, song, better, rememb, heart, start, bet...
6    [read, news, today, lucki, grade, news, laugh,...
7    [blackbird, sing, dead, night, break, wing, le...
8    [like, pig, cri, sit, cornflak, wait, come, co...
9    [look, lone, peopl, look, lone, peopl, eleanor...
Name: Lyrics, dtype: object

In [65]:
#displaying the most common tokens in the dataset
from collections import Counter
from itertools import *

text_word_frequency = Counter(chain.from_iterable(processed_lyrics))
text_word_frequency.most_common()[:20]

[('love', 3098),
 ('know', 2766),
 ('like', 2363),
 ('babi', 1631),
 ('come', 1575),
 ('want', 1287),
 ('time', 1259),
 ('feel', 1205),
 ('caus', 1151),
 ('girl', 997),
 ('wanna', 953),
 ('tell', 941),
 ('night', 854),
 ('gonna', 851),
 ('look', 845),
 ('think', 832),
 ('need', 814),
 ('right', 782),
 ('go', 751),
 ('away', 719)]

In [70]:
#display the least frequent tokens in the dataset
text_word_frequency.most_common()[-20:]

[('gimmick', 1),
 ('crumb', 1),
 ('loxvill', 1),
 ('jada', 1),
 ('jackass', 1),
 ('knoxvill', 1),
 ('thousandair', 1),
 ('outrun', 1),
 ('distribut', 1),
 ('fluid', 1),
 ('beefin', 1),
 ('relatin', 1),
 ('jason', 1),
 ('scrappin', 1),
 ('olympus', 1),
 ('psychiatrist', 1),
 ('uninspir', 1),
 ('handout', 1),
 ('welfar', 1),
 ('backstabbin', 1)]

### LDA Model Training ###

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_lyrics)

In [71]:
#setting the threshold of what to incorporate in the the training, 
dictionary.filter_extremes(no_below=5, no_above=0.3)

#the no_below is absolute frequency, no_above is percentage of of documents(songs) that has the token

In [96]:
#Bag-of-words transformation
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_lyrics]

In [73]:
#testing on the same song https://genius.com/Lady-gaga-and-blackpink-sour-candy-lyrics
bow_doc_342 = bow_corpus[342]

for i in range(len(bow_doc_342)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_342[i][0], 
                                                     dictionary[bow_doc_342[i][0]], 
                                                     bow_doc_342[i][1]))

Word 4 ("light") appears 2 time.
Word 26 ("hard") appears 4 time.
Word 65 ("littl") appears 2 time.
Word 117 ("turn") appears 2 time.
Word 119 ("break") appears 1 time.
Word 121 ("eye") appears 1 time.
Word 164 ("mean") appears 1 time.
Word 182 ("wanna") appears 1 time.
Word 184 ("sweet") appears 2 time.
Word 195 ("real") appears 1 time.
Word 271 ("close") appears 1 time.
Word 333 ("insid") appears 6 time.
Word 346 ("bite") appears 8 time.
Word 412 ("tast") appears 1 time.
Word 443 ("mess") appears 1 time.
Word 454 ("outsid") appears 4 time.
Word 538 ("fake") appears 1 time.
Word 566 ("crazi") appears 2 time.
Word 579 ("nice") appears 1 time.


In [104]:
#training the model with 10 topics
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [99]:
#print out the most prominent tokens for each topic
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"bodi" + 0.024*"gonna" + 0.016*"hear" + 0.014*"touch" + 0.014*"need" + 0.012*"girl" + 0.011*"leav" + 0.010*"heart" + 0.009*"bell" + 0.009*"talk"
Topic: 1 
Words: 0.023*"roll" + 0.019*"nigga" + 0.018*"leav" + 0.016*"face" + 0.016*"babe" + 0.016*"play" + 0.015*"girl" + 0.014*"bring" + 0.014*"okay" + 0.011*"hous"
Topic: 2 
Words: 0.039*"heart" + 0.018*"break" + 0.017*"dream" + 0.015*"work" + 0.015*"tonight" + 0.014*"thing" + 0.013*"mean" + 0.012*"world" + 0.012*"walk" + 0.011*"chang"
Topic: 3 
Words: 0.031*"stand" + 0.021*"gonna" + 0.020*"away" + 0.020*"rain" + 0.017*"shoot" + 0.014*"happi" + 0.012*"free" + 0.011*"place" + 0.011*"ladi" + 0.011*"guess"
Topic: 4 
Words: 0.040*"gonna" + 0.036*"away" + 0.035*"good" + 0.025*"thing" + 0.022*"gimm" + 0.022*"wait" + 0.016*"hold" + 0.010*"girl" + 0.010*"littl" + 0.010*"babe"
Topic: 5 
Words: 0.020*"danc" + 0.019*"wanna" + 0.018*"forev" + 0.015*"eye" + 0.013*"word" + 0.013*"beat" + 0.012*"lose" + 0.012*"believ" + 0.011*"fight

In [77]:
#testing it on the same song https://genius.com/Lady-gaga-and-blackpink-sour-candy-lyrics
for index, score in sorted(lda_model[bow_corpus[342]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8807077407836914	 
Topic: 0.052*"life" + 0.022*"light" + 0.018*"say" + 0.012*"good" + 0.011*"world" + 0.010*"river" + 0.010*"thing" + 0.010*"woah" + 0.010*"mind" + 0.010*"away"

Score: 0.10068244487047195	 
Topic: 0.031*"stand" + 0.021*"gonna" + 0.020*"away" + 0.020*"rain" + 0.017*"shoot" + 0.014*"happi" + 0.012*"free" + 0.011*"place" + 0.011*"ladi" + 0.011*"guess"


### LDA Visualization ###

In [ ]:
!pip install pyldavis

In [ ]:
import pickle
import pyLDAvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [ ]:
import pyLDAvis.gensim_models as gensimvis

In [78]:
pyLDAvis.enable_notebook()

In [79]:
LDAvis_prepared = gensimvis.prepare(lda_model, bow_corpus, dictionary)

In [82]:
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.098125 -0.012614       1        1  14.204740
7     -0.006757  0.004347       2        1  12.758183
2     -0.024142  0.051741       3        1  12.328874
6      0.020906  0.011613       4        1  11.453757
5     -0.011570  0.053607       5        1   9.148444
4     -0.063698 -0.057920       6        1   9.042272
3     -0.044741 -0.056126       7        1   8.666134
8     -0.059641  0.023524       8        1   8.538442
0      0.027098  0.019644       9        1   7.091871
1      0.064421 -0.037815      10        1   6.767283, topic_info=      Term        Freq       Total Category  logprob  loglift
518  nigga  687.000000  687.000000  Default  30.0000  30.0000
437  bitch  507.000000  507.000000  Default  29.0000  29.0000
123   life  686.000000  686.000000  Default  28.0000  28.0000
313  gonna  850.000000  850.000000  Default  27.0000  27.0000
521   fuck  522.000000  522.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
44    need   47.528751  775.024443  Topic10  -4.5910  -0.0985
4    light   36.847234  443.958231  Topic10  -4.8456   0.2041
313  gonna   37.686426  850.120979  Topic10  -4.8231  -0.4230
75   heart   30.741357  677.147409  Topic10  -5.0267  -0.3992
521   fuck   28.902142  522.391600  Topic10  -5.0884  -0.2014

[587 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
269       1  0.576141  actin
269       2  0.153637  actin
269       3  0.038409  actin
269       4  0.076819  actin
269       5  0.038409  actin
...     ...       ...    ...
249       6  0.098917  young
249       7  0.024729  young
249       8  0.118700  young
249       9  0.069242  young
249      10  0.034621  young

[3490 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 8, 3, 7, 6, 5, 4, 9, 1, 2])

### Recommender ###

In [ ]:
doc_lda = lda_model[bow_corpus]

In [ ]:
from gensim import similarities

In [103]:
index = similarities.MatrixSimilarity(doc_lda)

In [84]:
def song_recommender(url):
    songs_checked = 0
    for i in range(len(lyrics_df)):
        recommendation_scores = []
        if lyrics_df.iloc[i,0] == url:
            lda_vectors = doc_lda[i]
            sims = index[lda_vectors]
            sims = list(enumerate(sims))
            for sim in sims:
                song_num = sim[0]
                recommendation_score = [lyrics_df.iloc[song_num, 0], sim[1]]
                recommendation_scores.append(recommendation_score)
            
            recommendation = sorted(recommendation_scores, key=lambda x: x[1], reverse=True)
            print("Your song's most prominent tokens are:")
            article_tokens = bow_corpus[i]
            sorted_tokens = sorted(article_tokens, key=lambda x: x[1], reverse=True)
            sorted_tokens_10 = sorted_tokens[:10]
            for i in range(len(sorted_tokens_10)):
                print("Word {} (\"{}\") appears {} time(s).".format(sorted_tokens_10[i][0], 
                                                             dictionary[sorted_tokens_10[i][0]], 
                                                             sorted_tokens_10[i][1]))
            print('-----')
            print("Your song's most prominant topic is:")
            print(lda_model.print_topic(max(lda_vectors, key=lambda item: item[1])[0]))
            print('-----')
            print('Here are your recommendations for "{}":'.format(url))
            display(recommendation[1:21])
        
        else:
            songs_checked +=1
            
        if songs_checked == len(lyrics_df):
            book_suggestions = []
            print('Sorry, but it looks like "{}" is not available.'.format(url))
           

In [85]:
song_recommender('https://genius.com/The-beatles-yesterday-lyrics')

Your song's most prominent tokens are:
Word 51 ("yesterday") appears 9 time(s).
Word 36 ("away") appears 3 time(s).
Word 37 ("believ") appears 3 time(s).
Word 38 ("easi") appears 2 time(s).
Word 39 ("game") appears 2 time(s).
Word 42 ("hide") appears 2 time(s).
Word 43 ("long") appears 2 time(s).
Word 44 ("need") appears 2 time(s).
Word 45 ("place") appears 2 time(s).
Word 46 ("play") appears 2 time(s).
-----
Your song's most prominant topic is:
0.040*"gonna" + 0.036*"away" + 0.035*"good" + 0.025*"thing" + 0.022*"gimm" + 0.022*"wait" + 0.016*"hold" + 0.010*"girl" + 0.010*"littl" + 0.010*"babe"
-----
Here are your recommendations for "https://genius.com/The-beatles-yesterday-lyrics":


[['https://genius.com/Madonna-hung-up-lyrics', 1.0],
 ['https://genius.com/Mariah-carey-fantasy-bad-boy-remix-lyrics', 1.0],
 ['https://genius.com/Ac-dc-big-balls-lyrics', 1.0],
 ['https://genius.com/Drake-gods-plan-lyrics', 1.0],
 ['https://genius.com/U2-with-or-without-you-lyrics', 1.0],
 ['https://genius.com/Abba-gimme-gimme-gimme-a-man-after-midnight-lyrics',
  1.0],
 ['https://genius.com/Fleetwood-mac-landslide-lyrics', 1.0],
 ['https://genius.com/Bee-gees-too-much-heaven-lyrics', 1.0],
 ['https://genius.com/Britney-spears-gimme-more-lyrics', 1.0],
 ['https://genius.com/Shania-twain-im-gonna-getcha-good-lyrics', 1.0],
 ['https://genius.com/Neil-diamond-brother-loves-travelling-salvation-show-lyrics',
  1.0],
 ['https://genius.com/Dire-straits-so-far-away-lyrics', 1.0],
 ['https://genius.com/Sonny-and-cher-i-got-you-babe-lyrics', 1.0],
 ['https://genius.com/Stevie-wonder-dont-you-worry-bout-a-thing-lyrics', 1.0],
 ['https://genius.com/Linda-ronstadt-youre-no-good-lyrics', 1.0],
 ['

In [86]:
song_recommender('https://genius.com/Lady-gaga-and-blackpink-sour-candy-lyrics')

Your song's most prominent tokens are:
Word 346 ("bite") appears 8 time(s).
Word 333 ("insid") appears 6 time(s).
Word 26 ("hard") appears 4 time(s).
Word 454 ("outsid") appears 4 time(s).
Word 4 ("light") appears 2 time(s).
Word 65 ("littl") appears 2 time(s).
Word 117 ("turn") appears 2 time(s).
Word 184 ("sweet") appears 2 time(s).
Word 566 ("crazi") appears 2 time(s).
Word 119 ("break") appears 1 time(s).
-----
Your song's most prominant topic is:
0.039*"heart" + 0.018*"break" + 0.017*"dream" + 0.015*"work" + 0.015*"tonight" + 0.014*"thing" + 0.013*"mean" + 0.012*"world" + 0.012*"walk" + 0.011*"chang"
-----
Here are your recommendations for "https://genius.com/Lady-gaga-and-blackpink-sour-candy-lyrics":


[['https://genius.com/Linkin-park-one-more-light-lyrics', 0.9968595],
 ['https://genius.com/Elvis-presley-dont-be-cruel-lyrics', 0.9962783],
 ['https://genius.com/Rod-stewart-young-turks-lyrics', 0.98471946],
 ['https://genius.com/Aerosmith-amazing-lyrics', 0.9744961],
 ['https://genius.com/Phil-collins-in-the-air-tonight-lyrics', 0.9725146],
 ['https://genius.com/George-strait-i-cross-my-heart-lyrics', 0.9712481],
 ['https://genius.com/Garth-brooks-if-tomorrow-never-comes-lyrics', 0.9704274],
 ['https://genius.com/Stevie-wonder-sir-duke-lyrics', 0.9664204],
 ['https://genius.com/Coldplay-the-scientist-lyrics', 0.9616801],
 ['https://genius.com/Bryan-adams-heaven-lyrics', 0.95439106],
 ['https://genius.com/The-beatles-blackbird-lyrics', 0.95062566],
 ['https://genius.com/Bee-gees-stayin-alive-lyrics', 0.9450908],
 ['https://genius.com/Alicia-keys-unthinkable-remix-lyrics', 0.92848206],
 ['https://genius.com/The-beatles-eleanor-rigby-lyrics', 0.9246526],
 ['https://genius.com/Elton-john

In [87]:
song_recommender("https://genius.com/Eminem-lose-yourself-lyrics")

Your song's most prominent tokens are:
Word 71 ("better") appears 16 time(s).
Word 31 ("shoot") appears 9 time(s).
Word 124 ("moment") appears 8 time(s).
Word 85 ("blow") appears 7 time(s).
Word 1 ("chanc") appears 6 time(s).
Word 7 ("music") appears 6 time(s).
Word 213 ("miss") appears 6 time(s).
Word 406 ("lose") appears 6 time(s).
Word 53 ("grow") appears 4 time(s).
Word 123 ("life") appears 3 time(s).
-----
Your song's most prominant topic is:
0.023*"roll" + 0.019*"nigga" + 0.018*"leav" + 0.016*"face" + 0.016*"babe" + 0.016*"play" + 0.015*"girl" + 0.014*"bring" + 0.014*"okay" + 0.011*"hous"
-----
Here are your recommendations for "https://genius.com/Eminem-lose-yourself-lyrics":


[['https://genius.com/Eric-clapton-lay-down-sally-lyrics', 0.90180737],
 ['https://genius.com/Flo-rida-low-lyrics', 0.88613725],
 ['https://genius.com/Gloria-estefan-and-miami-sound-machine-1-2-3-lyrics',
  0.88204443],
 ['https://genius.com/The-rolling-stones-paint-it-black-lyrics', 0.8649461],
 ['https://genius.com/Beyonce-partition-lyrics', 0.84398973],
 ['https://genius.com/Def-leppard-rocket-lyrics', 0.81760895],
 ['https://genius.com/George-michael-faith-lyrics', 0.817442],
 ['https://genius.com/Barry-white-basketball-jones-lyrics', 0.81722224],
 ['https://genius.com/Garth-brooks-the-thunder-rolls-lyrics', 0.8171139],
 ['https://genius.com/Hercules-lillias-white-cheryl-freeman-lachanze-roz-ryan-vaneese-thomas-and-tawatha-agee-zero-to-hero-lyrics',
  0.81478715],
 ['https://genius.com/Bryan-adams-empty-spaces-what-shall-we-do-now-lyrics',
  0.8084644],
 ['https://genius.com/Michael-jackson-smooth-criminal-lyrics', 0.80560404],
 ['https://genius.com/Pink-floyd-another-brick-in-the-